In [12]:
import os
import isce
import asf_search as asf
from iscesys.Component.ProductManager import ProductManager
import sys
sys.path.append('/u/trappist-r0/colespeed/work/coseis/scripts/')  # Add the directory to the system path
import coseis
from osgeo import gdal
import pandas as pd
import geopandas as gpd
from shapely import wkt
import importlib

### Load in the earthquake epicenters

In [18]:
df = pd.read_csv('/u/trappist-r0/colespeed/work/coseis/earthquakes/earthquakes_since_2014.csv')

# Convert the 'geometry' column from WKT to shapely geometries (including Z coordinates)
df['geometry'] = df['geometry'].apply(wkt.loads)

# Create a GeoDataFrame from the DataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.set_crs(epsg=4326, inplace=True)
gdf.head()

,place,depth,mag,geometry,date,time
0,"48 km W of Illapel, Chile",22.44,8.3,POINT Z (-71.67440 -31.57290 22.44000),9/16/15,54:32.9
1,"67 km NNE of Bharatpur, Nepal",8.22,7.8,POINT Z (84.73140 28.23050 8.22000),4/25/15,11:25.9
2,"Pazarcik earthquake, Kahramanmaras earthquake ...",10.00,7.8,POINT Z (37.01430 37.22560 10.00000),2/6/23,17:34.3
3,"27 km SSE of Muisne, Ecuador",20.59,7.8,POINT Z (-79.92180 0.38190 20.59000),4/16/16,58:37.0
4,"53 km NNE of Amberley, New Zealand",15.11,7.8,POINT Z (173.05400 -42.73730 15.11000),11/13/16,02:56.3


### View earthquakes over magnitude 7.5

In [17]:
mag75 = gdf[gdf['mag'] >= 7.5]
mag75.explore()

### Set working directory

In [4]:
# Make base directory (if not already made)
base_directory = '/u/trappist-r0/colespeed/work/coseis/earthquakes'
os.makedirs(base_directory, exist_ok=True)

# Make earthquake directory (if not already made)
eq_dir= 'Turkiye_20240206'  # Update this to automatically take the USGS title from the metadata
eq_dir_path = os.path.join(base_directory, eq_dir)
os.makedirs(eq_dir_path, exist_ok=True)

track_dir = os.path.join(eq_dir_path, 'D021') # Automate the naming of the directories
os.makedirs(track_dir, exist_ok=True)
os.chdir(track_dir)

# Verify the current working directory
print("Current working directory:", os.getcwd())

Current working directory: /u/trappist-r0/colespeed/work/coseis/earthquakes/Turkiye_20240206/D021


### Find earthquakes

In [5]:
importlib.reload(coseis)
ascending_json, descending_json = coseis.main_historic('20230205', '20230207')

Fetching historic earthquake data from 20230205 to 20230207...
2024-12-21 01:41:54,438 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): earthquake.usgs.gov:443
2024-12-21 01:41:54,845 - urllib3.connectionpool - DEBUG - https://earthquake.usgs.gov:443 "GET /fdsnws/event/1/query?format=geojson&starttime=20230205&endtime=20230207&minmagnitude=6.0&maxdepth=30.0 HTTP/1.1" 200 1234
Checking for significant earthquakes...
2024-12-21 01:41:54,851 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): raw.githubusercontent.com:443
2024-12-21 01:41:54,865 - urllib3.connectionpool - DEBUG - https://raw.githubusercontent.com:443 "GET /OSGeo/PROJ/refs/heads/master/docs/plot/data/coastline.geojson HTTP/1.1" 200 92593
Found 1 significant earthquakes.
[{'mag': 7.8, 'place': 'Pazarcik earthquake, Kahramanmaras earthquake sequence', 'time': 1675646254342, 'updated': 1727854286861, 'tz': None, 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/us6000jllz', 'deta

### Set working directory

In [6]:
# # Make base directory (if not already made)
# base_directory = '/u/trappist-r0/colespeed/work/coseis/earthquakes'
# os.makedirs(base_directory, exist_ok=True)

# # Make earthquake directory (if not already made)
# eq_dir= 'Turkiye_20240206'  # Update this to automatically take the USGS title from the metadata
# eq_dir_path = os.path.join(base_directory, eq_dir)
# os.makedirs(eq_dir_path, exist_ok=True)

# # Verify the current working directory
# print("Current working directory:", os.getcwd())

### Make directories for ASCENDING and DESCENDING tracks

In [7]:
# track_dir = os.path.join(eq_dir_path, 'A014') # Automate the naming of the directories
# os.makedirs(track_dir, exist_ok=True)
# os.chdir(track_dir)

# # Verify the current working directory
# print("Current working directory:", os.getcwd())

In [8]:
print(f'Ascending JSON: {ascending_json}')
print(f'Descending JSON: {descending_json}')

Ascending JSON: {'reference-scenes': ['S1A_IW_SLC__1SDV_20230209T153357_20230209T153425_047161_05A88A_BB1A', 'S1A_IW_SLC__1SDV_20230209T153423_20230209T153450_047161_05A88A_9FEC'], 'secondary-scenes': ['S1A_IW_SLC__1SDV_20230128T153358_20230128T153426_046986_05A2B6_1629', 'S1A_IW_SLC__1SDV_20230128T153424_20230128T153451_046986_05A2B6_00C8'], 'frame-id': -1, 'estimate-ionosphere-delay': True, 'compute-solid-earth-tide': True, 'output-resolution': 30, 'unfiltered-coherence': True, 'goldstein-filter-power': 0.5, 'dense-offsets': True, 'wrapped-phase-layer': True, 'esd-coherence-threshold': -1}
Descending JSON: {'reference-scenes': ['S1A_IW_SLC__1SDV_20230210T033426_20230210T033454_047168_05A8CD_FAA6', 'S1A_IW_SLC__1SDV_20230210T033451_20230210T033518_047168_05A8CD_E5B0'], 'secondary-scenes': ['S1A_IW_SLC__1SDV_20230129T033427_20230129T033455_046993_05A2FE_6FF2', 'S1A_IW_SLC__1SDV_20230129T033452_20230129T033519_046993_05A2FE_BE0B'], 'frame-id': -1, 'estimate-ionosphere-delay': True, 'com

### Run DockerizedTopsApp

In [ ]:
import time
start_time = time.time()
!XLA_PYTHON_CLIENT_MEM_FRACTION=.20 taskset -c 62-94 nohup isce2_topsapp --reference-scenes S1A_IW_SLC__1SDV_20230210T033426_20230210T033454_047168_05A8CD_FAA6 S1A_IW_SLC__1SDV_20230210T033451_20230210T033518_047168_05A8CD_E5B0 --secondary-scenes S1A_IW_SLC__1SDV_20230129T033427_20230129T033455_046993_05A2FE_6FF2 S1A_IW_SLC__1SDV_20230129T033452_20230129T033519_046993_05A2FE_BE0B --frame-id -1 --estimate-ionosphere-delay True --esd-coherence-threshold -1 --compute-solid-earth-tide True --goldstein-filter-power 0.5 --output-resolution 30 --unfiltered-coherence True --dense-offsets True 

end_time = time.time()
elapsed_time = end_time - start_time

# Print the result
print(f"Script completed in {elapsed_time:.2f} seconds")  

nohup: ignoring input and appending output to ‘nohup.out’


### Produce Ascending track coseismic displacement product

In [73]:
import time
start_time = time.time()
!XLA_PYTHON_CLIENT_MEM_FRACTION=.20 taskset -c 62-94 nohup isce2_topsapp ++json 'ascending_group.json'
end_time = time.time()
elapsed_time = end_time - start_time

# Print the result
print(f"Script completed in {elapsed_time:.2f} seconds") 

nohup: ignoring input and appending output to ‘nohup.out’


Script completed in 3.93 seconds


### Download orbit files

In [8]:
!eof --search-path {slc_dir} --save-dir {orbits_dir} --force-asf

[12/03 21:23:30] [INFO download.py] Skipping /u/trappist-r0/colespeed/work/coseis/earthquakes/ridgecrest/data/slcs/S1B_IW_SLC__1SDV_20190710T014959_20190710T015026_017065_0201B8_069B, already have EOF file
[12/03 21:23:30] [INFO download.py] Skipping /u/trappist-r0/colespeed/work/coseis/earthquakes/ridgecrest/data/slcs/S1A_IW_SLC__1SDV_20190704T015023_20190704T015051_027961_03283A_D6EA, already have EOF file
[12/03 21:23:30] [INFO download.py] Skipping /u/trappist-r0/colespeed/work/coseis/earthquakes/ridgecrest/data/slcs/S1A_IW_SLC__1SDV_20190704T015049_20190704T015116_027961_03283A_191E, already have EOF file
[12/03 21:23:30] [INFO download.py] No Sentinel products found in directory /u/trappist-r0/colespeed/work/coseis/earthquakes/ridgecrest/data/slcs, exiting


### Create XML files
`TODO: Write function to construct the XML files specific to reference/secondary SLCs`

For now, I have just loaded them in manually.

### topsApp workflow

In [ ]:
# %%bash
# nohup topsApp.py > output.log 2>&1 &

In [82]:
!nohup isce2_topsapp  --reference-scenes S1B_IW_SLC__1SDV_20190710T014959_20190710T015026_017065_0201B8_069B --secondary-scenes S1A_IW_SLC__1SDV_20190704T015023_20190704T015051_027961_03283A_D6EA S1A_IW_SLC__1SDV_20190704T015049_20190704T015116_027961_03283A_191E --output-resolution 30 \
--esd-coherence-threshold -1 \
--estimate-ionosphere-delay True \
--goldstein-filter-power 0.5 \
--unfiltered-coherence True \
--dense-offsets True \
--frame-id -1 

nohup: ignoring input and appending output to ‘nohup.out’


### Pixel masking

In [ ]:
f1 = 'merged/filt_topophase.unw.conncomp.geo'
f2 = 'merged/filt_topophase_msk.unw.geo'
outfile = 'merged/filt_topophase_mskAlias.unw.geo'
#mask_int_alias_2.mask_unw(f1, f2, outfile)

None


ERROR 4: merged/filt_topophase_msk.unw.geo: No such file or directory


### Create wrapped tif

In [ ]:
!python /u/trappist-r0/colespeed/work/coseis/scripts/create_wrapped.py /u/trappist-r0/colespeed/work/coseis/earthquakes/ridgecrest/insar/merged/filt_topophase.unw.geo /u/trappist-r0/colespeed/work/coseis/earthquakes/ridgecrest/insar/merged/filt_topophase.tif

### Extract pixel dimensions and compute dense offsets in meters

In [63]:
def load_product(xml_name):
    """Load the product using Product Manager."""
    pm = ProductManager()
    pm.configure()
    return pm.loadProduct(xml_name)

reference_dir = os.path.join(insar_dir, 'reference')
xml_file = os.path.join(reference_dir, 'IW1.xml')
obj = load_product(xml_file)
burst = obj.bursts[0]
burstEnd = obj.bursts[-1]

In [64]:
import numpy as np

orbit = burst.orbit
peg = orbit.interpolateOrbit(burst.sensingMid, method='hermite')

# Sentinel-1 TOPS pixel spacing
Vs = np.linalg.norm(peg.getVelocity())   #satellite speed

In [ ]:
from isceobj.Planet.Planet import Planet

azimuthPixelSize = Vs * burst.azimuthTimeInterval
earthRadius = Planet(pname='Earth').ellipsoid.pegRadCur

NameError: name 'Planet' is not defined

In [66]:
sensingMid = obj.sensingMid
print(sensingMid)

2019-07-04 01:50:49.718705


In [67]:
burst.terrainHeight

63.748288

In [9]:
burst.parameter_list

(Parameter('numberOfLines', public_name='number of lines', default=None, type=int, units=None, mandatory=True, private=False),
 Parameter('numberOfSamples', public_name='number of samples', default=None, type=int, units=None, mandatory=True, private=False),
 Parameter('startingRange', public_name='starting range', default=None, type=float, units=None, mandatory=True, private=False),
 Parameter('sensingStart', public_name='sensing start', default=None, type=datetimeType, units=None, mandatory=True, private=False),
 Parameter('sensingStop', public_name='sensing stop', default=None, type=datetimeType, units=None, mandatory=True, private=False),
 Parameter('burstStartUTC', public_name='burst start utc', default=None, type=datetimeType, units=None, mandatory=True, private=False),
 Parameter('burstStopUTC', public_name='burst stop utc', default=None, type=datetimeType, units=None, mandatory=True, private=False),
 Parameter('trackNumber', public_name='track number', default=None, type=int, un

In [19]:
# Re, h_sat = float(meta['EARTH_RADIUS']), float(meta['HEIGHT'])
# pixel_size = float(meta['AZIMUTH_PIXEL_SIZE']) * Re / (Re + h_sat)
# print('Ground azimuth pixel size: {:.2f} m'.format(pixel_size))

In [28]:
# Input raster file
dense_offsets = 'merged/filt_dense_offsets.bil.geo'

# Output raster files
output_range_m = "filt_dense_offsets_range_m.tif"
output_azimuth_m = "filt_dense_offsets_azimuth_m.tif"

rangePixelSize = burst.rangePixelSize 
azimuthPixelSize = 14.02 # placeholder value for now

# Open the input raster
dataset = gdal.Open(dense_offsets)
if not dataset:
    raise RuntimeError(f"Failed to open file: {dense_offsets}")

# Read the first band and process it
range = dataset.GetRasterBand(2).ReadAsArray()
range_m = range * rangePixelSize

# Create output raster for Band 1
driver = gdal.GetDriverByName("GTiff")
out_ds_band1 = driver.Create(
    output_range_m, 
    dataset.RasterXSize, 
    dataset.RasterYSize, 
    1,  # Number of bands
    gdal.GDT_Float32
)
out_ds_band1.SetGeoTransform(dataset.GetGeoTransform())
out_ds_band1.SetProjection(dataset.GetProjection())
out_ds_band1.GetRasterBand(1).WriteArray(range_m)
out_ds_band1 = None  # Close the file

# Read the second band and process it
azimuth = dataset.GetRasterBand(1).ReadAsArray()
azimuth_m = azimuth * azimuthPixelSize

# Create output raster for Band 2
driver = gdal.GetDriverByName("GTiff")
out_ds_band2 = driver.Create(
    output_azimuth_m, 
    dataset.RasterXSize, 
    dataset.RasterYSize, 
    1,  # Number of bands
    gdal.GDT_Float32
)
out_ds_band2.SetGeoTransform(dataset.GetGeoTransform())
out_ds_band2.SetProjection(dataset.GetProjection())
out_ds_band2.GetRasterBand(1).WriteArray(azimuth_m)
out_ds_band2 = None  # Close the file

# Clean up
dataset = None

print("Processing complete. Files saved:")
print(f" - {output_range_m}")
print(f" - {output_azimuth_m}")

Processing complete. Files saved:
 - filt_dense_offsets_range_m.tif
 - filt_dense_offsets_azimuth_m.tif


In [11]:
!gdalinfo /u/trappist-r0/colespeed/work/coseis/earthquakes/ridgecrest/insar/merged/filt_dense_offsets.bil.geo

Driver: ISCE/ISCE raster
Files: /u/trappist-r0/colespeed/work/coseis/earthquakes/ridgecrest/insar/merged/filt_dense_offsets.bil.geo
       /u/trappist-r0/colespeed/work/coseis/earthquakes/ridgecrest/insar/merged/filt_dense_offsets.bil.geo.xml
Size is 11579, 6748
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (-119.457222222222228,36.880833333333335)
Pixel Size = (0.000277777777778,-0.000277777777778)
Corner Coordinates:
Upper Left  (-119.4572222,  36.8808333) (119d27'26.